In [3]:
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.embedder.openai import OpenAIEmbedder

#------------------ settings ------------------
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen-plus-latest'
embedding_model_id = 'text-embedding-v3'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

settings = local_settings
#------------------ settings ------------------

In [4]:
vector_db = LanceDb(
    table_name="contact_info_vectors",
    uri="tmp/lancedb",
    search_type=SearchType.hybrid,
    embedder=OpenAIEmbedder(id=embedding_model_id,api_key=api_key,base_url=base_url, dimensions=1024),
)

INFO Creating table: contact_info_vectors

In [6]:
contact_facts = """
武钢一、四烧结节能环保提升改造烟气净化项目
武钢五烧结节能环保提升改造余热发电项目
武钢二三烧结整合大修改造烟气净化项目
"""

In [19]:
from agno.document.base import Document
from agno.knowledge.document import DocumentKnowledgeBase
from agno.tools.reasoning import ReasoningTools
# Load documents from the data/docs directory
documents = [Document(content=contact_facts)]


In [8]:
knowledge_base = DocumentKnowledgeBase(
    documents=documents,
    vector_db=vector_db,
)

In [9]:
# Load the knowledge base
knowledge_base.load(recreate=False)

INFO Loading knowledge base

INFO Inserting 1 documents

INFO Added 1 documents to knowledge base

In [ ]:
from pymongo import MongoClient
import pandas as pd
mongo_uri = "mongodb://192.168.48.132:27017/"
def get_mongo_client(uri):
    try:
        return MongoClient(uri)
    except Exception as e:
        print(f"连接失败: {str(e)}")

# 执行MongoDB查询
def execute_mongo_query(project_name:str):
    try:
        client = get_mongo_client(mongo_uri)
        db = client['equipment_db']
        collection = db['equipment_collection_copy']
        
        # 执行查询
        try:
            results = list(collection.find({"project_name":project_name
            }).sort({
                'project_name': 1,
                'table_index': 1,
                'table_row_index': 1
            }))
            
            if not results:
                print("没有找到匹配的文档")
                return None, "没有找到匹配的记录"
                
        except Exception as e:
            print(f"查询失败: {str(e)}")
            return None, f"查询失败: {str(e)}"
        
        df = pd.json_normalize(results)
         # 定义需要显示的字段及其对应的中文表头
        display_columns = {
            'device_name': '设备名称',
            'unit_price': '单价',
            'quantity': '数量',
            'total_price': '总价',
            'price_unit': '价格单位',
            'unit': '单位',
            'project_name': '项目名称',
            'contact_name': '合同名称',
            'subitem_name': '子项名称',
            'contract_number': '合同号',
            'contact_type': '合同类型',
            'manufacturer': '制造商',
            'table_row_index': '表格行',
            'table_index': '表格索引'
        }
        
        # 筛选出指定的字段
        available_columns = [col for col in display_columns.keys() if col in df.columns]
        if not available_columns:
            print("查询结果中不包含指定的字段")
            return None, "查询结果中不包含指定的字段"
        
        df = df[available_columns]
        
        # 重命名列为中文表头
        df = df.rename(columns={col: display_columns[col] for col in available_columns})
        
        return df
    
    except Exception as e:
        return None, f"查询错误: {str(e)}"

In [30]:
client = get_mongo_client(mongo_uri)

In [31]:
client

MongoClient(host=['192.168.0.132:27017'], document_class=dict, tz_aware=False, connect=True)

In [38]:
df = execute_mongo_query('武钢一、四烧结节能环保提升改造烟气净化项目')
df.to_json(orient='records', force_ascii=False)

'[{"设备名称":"环境除尘器","单价":"620600","数量":"1","总价":"620600","价格单位":"元","单位":"台","项目名称":"武钢一、四烧结节能环保提升改造烟气净化项目","子项名称":"除尘系统","合同号":"WGYQJH1-2021-12","制造商":null,"表格行":null,"表格索引":"2"},{"设备名称":"卸料除尘器","单价":"295480","数量":"1","总价":"295480","价格单位":"元","单位":"台","项目名称":"武钢一、四烧结节能环保提升改造烟气净化项目","子项名称":"除尘系统","合同号":"WGYQJH1-2021-12","制造商":null,"表格行":null,"表格索引":"2"},{"设备名称":"负压气力输送系统","单价":"624200","数量":"1","总价":"624200","价格单位":"元","单位":"套","项目名称":"武钢一、四烧结节能环保提升改造烟气净化项目","子项名称":"除尘系统","合同号":"WGYQJH1-2021-12","制造商":null,"表格行":null,"表格索引":"2"},{"设备名称":"总计：大写人民币 壹佰伍拾肆万零贰佰捌拾圆整（￥1540280元）。","单价":"总计：大写人民币 壹佰伍拾肆万零贰佰捌拾圆整（￥1540280元）。","数量":"总计：大写人民币 壹佰伍拾肆万零贰佰捌拾圆整（￥1540280元）。","总价":"总计：大写人民币 壹佰伍拾肆万零贰佰捌拾圆整（￥1540280元）。","价格单位":"元","单位":"总计：大写人民币 壹佰伍拾肆万零贰佰捌拾圆整（￥1540280元）。","项目名称":"武钢一、四烧结节能环保提升改造烟气净化项目","子项名称":"除尘系统","合同号":"WGYQJH1-2021-12","制造商":null,"表格行":null,"表格索引":"2"},{"设备名称":"设备本体","单价":"","数量":"","总价":"","价格单位":"元","单位":"","项目名称":"武钢一、四烧结节能环保提升改造烟气净化项目","子项名称":"除尘系统","合同号":"WGYQJH1-2021-12","制造商":"","表格行

In [40]:
from agno.tools import tool

@tool()
def get_contact_info_by_project_name(project_name:str)->str:
    '''
    Get the contact info by project name.
    
    Args:
        project_name (str): The name of the project.
        
    Returns:
        str: The contact info of the project.
    '''
    df = execute_mongo_query(project_name)
    return df.to_json(orient='records', force_ascii=False)

In [43]:
agent = Agent(
    model=OpenAILike(**settings),
    instructions=['根据知识库中存储的项目名称，选择最相关的项目返回','根据知识库中的标准项目名称，使用工具查询的相关的采购合同明细数据','查询合同详情的时候请列出所有数据，严禁遗漏任何条目','如果需要进行合同比对的时候，请按需查出所有项目后再进行比对'],
    knowledge=knowledge_base,
    tools=[ReasoningTools(add_instructions=True),get_contact_info_by_project_name],
    markdown=True,
    telemetry=False
)

In [46]:
response = agent.run(message='武钢一四烧结余热发电有哪些采购设备')

没有找到匹配的文档


ERROR    Error in tool 'get_contact_info_by_project_name': AttributeError("'tuple' object has no attribute         
         'to_json'")                                                                                               
         Traceback (most recent call last):                                                                        
           File "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\agno\tools\decorator.py",
         line 133, in sync_wrapper                                                                                 
             return func(*args, **kwargs)                                                                          
           File "C:\Users\11798\AppData\Local\Temp\ipykernel_5644\2303934300.py", line 15, in                      
         get_contact_info_by_project_name                                                                          
             return df.to_json(orient='records', force_ascii=False)                                                
                    ^^^^^^^^^^                                                                                     
         AttributeError: 'tuple' object has no attribute 'to_json'

WARNING  Could not run function get_contact_info_by_project_name(project_name=武钢4号烧结机余热发电项目)

ERROR    'tuple' object has no attribute 'to_json'                                                                 
         Traceback (most recent call last):                                                                        
           File "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\agno\tools\function.py", 
         line 600, in execute                                                                                      
             result = self.function.entrypoint(**arguments)                                                        
           File                                                                                                    
         "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_validate_cal
         l.py", line 38, in wrapper_function                                                                       
             return wrapper(*args, **kwargs)                                                                       
           File                                                                                                    
         "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_validate_cal
         l.py", line 111, in __call__                                                                              
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
           File "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\agno\tools\decorator.py",
         line 133, in sync_wrapper                                                                                 
             return func(*args, **kwargs)                                                                          
           File "C:\Users\11798\AppData\Local\Temp\ipykernel_5644\2303934300.py", line 15, in                      
         get_contact_info_by_project_name                                                                          
             return df.to_json(orient='records', force_ascii=False)                                                
                    ^^^^^^^^^^                                                                                     
         AttributeError: 'tuple' object has no attribute 'to_json'

没有找到匹配的文档


ERROR    Error in tool 'get_contact_info_by_project_name': AttributeError("'tuple' object has no attribute         
         'to_json'")                                                                                               
         Traceback (most recent call last):                                                                        
           File "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\agno\tools\decorator.py",
         line 133, in sync_wrapper                                                                                 
             return func(*args, **kwargs)                                                                          
           File "C:\Users\11798\AppData\Local\Temp\ipykernel_5644\2303934300.py", line 15, in                      
         get_contact_info_by_project_name                                                                          
             return df.to_json(orient='records', force_ascii=False)                                                
                    ^^^^^^^^^^                                                                                     
         AttributeError: 'tuple' object has no attribute 'to_json'

WARNING  Could not run function get_contact_info_by_project_name(project_name=武钢1号烧结机余热发电项目)

ERROR    'tuple' object has no attribute 'to_json'                                                                 
         Traceback (most recent call last):                                                                        
           File "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\agno\tools\function.py", 
         line 600, in execute                                                                                      
             result = self.function.entrypoint(**arguments)                                                        
           File                                                                                                    
         "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_validate_cal
         l.py", line 38, in wrapper_function                                                                       
             return wrapper(*args, **kwargs)                                                                       
           File                                                                                                    
         "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_validate_cal
         l.py", line 111, in __call__                                                                              
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
           File "c:\Users\11798\AppData\Local\Programs\Python\Python313\Lib\site-packages\agno\tools\decorator.py",
         line 133, in sync_wrapper                                                                                 
             return func(*args, **kwargs)                                                                          
           File "C:\Users\11798\AppData\Local\Temp\ipykernel_5644\2303934300.py", line 15, in                      
         get_contact_info_by_project_name                                                                          
             return df.to_json(orient='records', force_ascii=False)                                                
                    ^^^^^^^^^^                                                                                     
         AttributeError: 'tuple' object has no attribute 'to_json'

INFO Found 1 documents

INFO Found 1 documents

In [47]:
print(response.content)















根据知识库检索结果，**武钢1号/4号烧结机余热发电项目** 的采购设备信息未在现有知识库中存储。检索仅返回了相关项目名称列表，但未包含具体采购合同或设备明细。

建议：
1. 确认知识库是否包含完整的采购合同数据
2. 尝试通过项目编号（如EPC-2021-001类似格式）进一步定位
3. 联系武钢集团采购部门获取原始合同文件

现有公开信息仅能确认以下相关项目存在：
- 武钢一、四烧结节能环保提升改造烟气净化项目
- 武钢五烧结节能环保提升改造余热发电项目
- 武钢二三烧结整合大修改造烟气净化项目
